In [33]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import emoji
import re

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

In [34]:
os.listdir("reviewDatas")

['365dni.csv',
 'AfterWeCollided.csv',
 'Chhalaang.csv',
 'Deadpool.csv',
 'Dolittle.csv',
 'Emma..csv',
 'EnolaHolmes .csv',
 'freaky.csv',
 'Gisaengchung.csv',
 'greenland.csv',
 'Hamilton.csv',
 'holidate.csv',
 'homeAlone.csv',
 'JingleJangleAChristmasJourney.csv',
 'JiuJitsu.csv',
 'Loimpossible.csv',
 'LoveActually.csv',
 'Ludo.csv',
 'meganIsMissing.csv',
 'MonsterProblems.csv',
 'Mulan.csv',
 'Nokeunokeu.csv',
 'Rebecca.csv',
 'SooraraiPottru.csv',
 'Spider-ManIntotheSpider-Verse.csv',
 'TheNewMutants.csv',
 'TheTrialoftheChicago7.csv',
 'TheWitches.csv',
 'ThorRagnarok.csv',
 'Unhinged.csv']

In [35]:
path = os.listdir("reviewDatas")
def load_data(path):
    movie_review = []
    ma = movie_review.append
    for movie in path:
        data = pd.read_csv("reviewDatas\\"+movie, names = ["idx", "review"])
        data = data.iloc[1:,1:]
        review_list = []
        ra = review_list.append
        for i in range(len(data)):
            ra(data.iloc[i,0])
        ma(review_list)
    return movie_review

In [31]:
def preprocessing(review_list):
    multi_emoji = re.compile(r'[:]{2,}[\w_-]+')
    shortword = re.compile(r'\W*\b\w{1,2}\b')
    punc = re.compile(r'[\W_]+')
    first_upper = re.compile(r'\s*[A-Z][a-z0-9]+')
    digit = re.compile(r'\d+')

    token_list = []
    ta = token_list.append
    for i in range(len(review_list)):
        review_list[i] = multi_emoji.sub('',emoji.demojize(review_list[i]))
        review_list[i] = shortword.sub('',review_list[i])
        review_list[i] = punc.sub(' ',review_list[i])
        review_list[i] = first_upper.sub('',review_list[i])
        review_list[i] = digit.sub('',review_list[i])
        review_list[i] = shortword.sub('',review_list[i])
        review_list[i] = review_list[i].lower()
        review_list[i] = review_list[i].strip()

        #토큰화
        ta(re.split('\s+',review_list[i]))
    return token_list

In [30]:
#불용어 목록 만들기
def make_stopwords() :
    stop_words = set(stopwords.words("english"))
    google_sw = "a about above after again against all am an and any are aren't as at be because been before being below between both but by can't cannot could couldn't did didn't do does doesn't doing don't down during each few for from further had hadn't has hasn't have haven't having he he'd he'll he's her here here's hers herself him himself his how how's i i'd i'll i'm i've if in into is isn't it it's its itself let's me more most mustn't my myself no nor not of off on once only or other ought our ours ourselves out over own same shan't she she'd she'll she's should shouldn't so some such than that that's the their theirs them themselves then there there's these they they'd they'll they're they've this those through to too under until up very was wasn't we we'd we'll we're we've were weren't what what's when when's where where's which while who who's whom why why's with won't would wouldn't you you'd you'll you're you've your yours yourself yourselves"
    google_sw = set(google_sw.split(' '))
    no_upper_sw = set("arent cant couldnt didnt doesnt dont hadnt hasnt hed hes heres hows id ill im ive isnt lets mustnt shant shed shes shouldnt thats theres theyd theyll theyre theyve wasnt wed weve werent whats whens wheres whos whys wont wouldnt youd youll youre youve".split(' '))

    stop_words = stop_words | google_sw | no_upper_sw
    return stop_words

print(len(stop_words))

255


In [29]:
def elim_stopwords(all_token):
    stop_words = make_stopwords()
    for token_list in all_token:
        for i,li in enumerate(token_list):
            token_list[i] = [item for item in li if item not in stop_words]
    print("eliminating stopwords is done --> pos tagging")
    return all_token

In [28]:
def pos_tagging(all_token, min_words = 5):
    for i, token_list in enumerate(all_token):
        temp = []
        ta = temp.append
        for j, li in enumerate(token_list):
            if len(li) > min_words:
                ta(pos_tag(li))
        all_token[i] = temp
        if (i+1) % 3 == 0:
            print("progress = {}%".format((i+1) / len(all_token)*100))
    print("pos-tagging is done --> lemmatization")
    return all_token

In [26]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [14]:
def pipeline(data, min_words = 5):
    all_token = []
    aa = all_token.append
    for idx,review_list in enumerate(data):
        token_list = preprocessing(review_list)
        aa(token_list)
        if (idx+1) % 3 == 0:
            print("{}%".format((idx+1) / len(data) * 100), end = " ")
    print()
    print("preprocessing is done --> elimination stopwords")
        
    all_token = lemmatization(pos_tagging(elim_stopwords(all_token),min_words))
    
    return all_token

In [25]:
def lemmatization(all_token):
    n=WordNetLemmatizer()
    for i, token_list in enumerate(all_token):
        for j, review_token in enumerate(token_list):
            for k, token in enumerate(review_token):
                try:
                    review_token[k] = n.lemmatize(token[0],get_wordnet_pos(token[1]))
                except KeyError:
                    review_token[k] = n.lemmatize(token[0])
        if (i+1) % 3 == 0:
            print("progress = {}%".format((i+1) / len(all_token)*100))
    print("all process is done!!")
    return all_token

In [36]:
movie_review = load_data(path)

In [37]:
all_token = pipeline(movie_review,5)

10.0% 20.0% 30.0% 40.0% 50.0% 60.0% 70.0% 80.0% 90.0% 100.0% eliminating stopwords is done --> pos tagging
progress = 10.0%
progress = 20.0%
progress = 30.0%
progress = 40.0%
progress = 50.0%
progress = 60.0%
progress = 70.0%
progress = 80.0%
progress = 90.0%
progress = 100.0%
pos-tagging is done --> lemmatization
progress = 10.0%
progress = 20.0%
progress = 30.0%
progress = 40.0%
progress = 50.0%
progress = 60.0%
progress = 70.0%
progress = 80.0%
progress = 90.0%
progress = 100.0%
all process is done!!


In [45]:
for i in range(30):
    for j in range(len(all_token[i])):
        all_token[i][j] = " ".join(all_token[i][j])

In [47]:
for i in range(30):
    df = pd.DataFrame(all_token[i])
    df.to_csv(os.listdir("reviewDatas")[i])

## 전처리 방안

+ 이모티콘을 단어로 변환 후 중복된 이모티콘을 1개로 만들기   
+ 단어의 길이가 1에서 2인 것 제거    
+ 리뷰의 구두점들을 제거      
+ 첫번째 단어가 대문자거나 대문자로 되어있는 모든 단어들 제거   
+ 숫자 제거      
+ 제거되지 않은 대문자 모임 소문자화 후 양끝 공백제거

In [6]:
all_token = []
aa = all_token.append
for idx,review_list in enumerate(movie_review):
    token_list = preprocessing(review_list)
    aa(token_list)
    if (idx+1) % 3 == 0:
        print("{}%".format((idx+1) / len(movie_review) * 100), end = " ")

10.0% 20.0% 30.0% 40.0% 50.0% 60.0% 70.0% 80.0% 90.0% 100.0% 

all_token 리스트의 길이 30개(영화 수)    
all_token\[0\]는 첫번째 영화의 token_list  

## 불용어 처리

추가사항   
구글의 stopwords english와 합집합.   
리뷰에 의도적으로 어퍼스토로피를 빼고 작성한 경우 존재.   
stopwords가 작동을 하지 않음.    
따라서 적당히 단어가 되지 않는 선에서 stopwords 추가   

In [9]:
all_token = elim_stopwords(all_token)

# 제거해야 하는 리뷰들    

+ 첫번째 글자를 항상 대문자로 작성하였음...     
    
데드풀 938,"Deadpool, Was The Perfect Comic Book Movie, Ryan Reynolds Was Born To Play The Character, And The Movie Is A Very Good Different Kind Of Movie, It's Fun To Watch! Has Great Jokes It's Fun! Funny! Has Great Wallbreakers! Good Amount Of Action! You Don't Want The Movie To End It's Been A While Sense I Seen A Superhero That's This Good And Perfect! Ryan Reynolds Is Inspiring In This Movie Makes You Wanna Become An Actor So You Can Play A Character Like This! This Movie Deserves To Be On Imdbs Top Movies, It Can't Get Better Than That, Hoping To See More Deadpool In The Future, One Of The Best Superhero/Antiheros Ever! Movie Had A Simple But Fun Story!"    
    
기생충 759,"I'm An Indian.I Watched It After A Lot Of Recommendation,But It Never Fullfilled My Expectations.The Movie Shows The Division In Between Peoples Such As High Class And Low Class,It Presents In The Way Of A Comedy Drama.I'm Not Satisfied On The Movie Maybe Because Of I'd Watch Lot Of Indian Movies Which Has These Kind Of Story."     
    
Ludo 67 "A Nice Movie Which Has Everything In It. If Creators Cut Duration By 30 Minutes Then It's Became More Enjoyable."    
     
Mulan 1337,"I Dislike When People Compare The Mulan Live Action Movie To The Original Animated Movie There Two Totally Different   Movies.I Thought The Cinematography Was Absolutely Astounding,The Characters Where Amazing And The Special Effects Where Great."   
    
SooraraiPottru 287 "Good Direction Fabulous Acting Good Cinematography"    
    
스파이더맨 146,"Animation 10/10     
Story 10/10     
Direction 10/10     
Script 10/10     
Imagination 100/10Loved it...."     

TheTrialoftheChicago7 25, "Performances: 10/10 Pacing: 7/10 Plot: 9/10 Writing: 10/10 Cinematography: 7/10 Soundtrack: 7/10        Enjoyability: 8/10"         
    
나크나로크 907,Niiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiicccccccccccccccccccccccccccceeeeeeeeeeeeeeeeeeeeeeeeeeee     

# Pos-tagging    

품사 태깅

In [11]:
all_token = pos_tagging(all_token)

progress = 10.0%
progress = 20.0%
progress = 30.0%
progress = 40.0%
progress = 50.0%
progress = 60.0%
progress = 70.0%
progress = 80.0%
progress = 90.0%
progress = 100.0%


# Stemming and Lemmatization

감성단어는 보통 형용사가 많기 때문에    
Stemming은 과한 일반화가 될 수 있으므로 시행하지 않음   
pos-tagging(품사 태깅) 후 Lemmatization만 시행

In [14]:
all_token[8][759]

[('never', 'RB'),
 ('seen', 'VBN'),
 ('movie', 'NN'),
 ('every', 'DT'),
 ('little', 'JJ'),
 ('detail', 'NN'),
 ('dialogue', 'NN'),
 ('object', 'NN'),
 ('foreshadows', 'VBZ'),
 ('things', 'NNS'),
 ('come', 'VBP'),
 ('fabulous', 'JJ'),
 ('way', 'NN'),
 ('made', 'VBD'),
 ('film', 'NN'),
 ('entertaining', 'VBG'),
 ('lots', 'NNS'),
 ('suspense', 'JJ'),
 ('end', 'NN')]

In [16]:
all_token = lemmatization(all_token)

progress = 10.0%
progress = 20.0%
progress = 30.0%
progress = 40.0%
progress = 50.0%
progress = 60.0%
progress = 70.0%
progress = 80.0%
progress = 90.0%
progress = 100.0%


all_token\[0\~29\] --> 영화 0번부터 29번까지    
all_token\[0\]\[0\~k\] --> 0번 영화의 0~k번째 리뷰의 토큰 리스트

### Counter를 이용한 영화별 단어 수 체크

In [17]:
# 25 index가 리뷰 수가 가장 적음
from collections import Counter

for i in range(1):
    for j in range(len(all_token[25])):
        counter = Counter(all_token[i][j])
        print(counter)

Counter({'sexy': 2, 'even': 1, 'know': 1, 'start': 1, 'nothing': 1, 'straight': 1, 'materialism': 1, 'plot': 1, 'normal': 1, 'chemistry': 1, 'two': 1, 'lead': 1, 'allure': 1, 'act': 1, 'natural': 1, 'say': 1, 'watch': 1, 'hot': 1, 'horny': 1, 'good': 1})
Counter({'movie': 25, 'woman': 12, 'make': 9, 'fantasy': 8, 'love': 7, 'go': 7, 'take': 7, 'give': 6, 'try': 6, 'like': 6, 'sexual': 6, 'porn': 6, 'male': 5, 'girl': 5, 'play': 5, 'get': 5, 'many': 5, 'fall': 4, 'completely': 4, 'mind': 4, 'understand': 4, 'first': 4, 'place': 4, 'normal': 4, 'see': 4, 'think': 4, 'young': 3, 'one': 3, 'two': 3, 'look': 3, 'film': 3, 'thing': 3, 'lot': 3, 'scenario': 3, 'seem': 3, 'let': 3, 'already': 3, 'feel': 3, 'plot': 3, 'bizarre': 3, 'accept': 3, 'tease': 3, 'end': 3, 'wrong': 3, 'still': 3, 'female': 3, 'also': 3, 'desire': 3, 'turn': 3, 'man': 3, 'live': 3, 'silly': 3, 'enjoy': 3, 'traditional': 3, 'aim': 3, 'men': 3, 'day': 2, 'successful': 2, 'handsome': 2, 'kidnaps': 2, 'year': 2, 'eventuall